Try out the gnn model


In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import numpy as np

ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
adata_rna = sc.read_h5ad("/lustre/groups/ml01/workspace/eirini.giannakoulia/datasets/V11L12-038/V11L12-038_A1/V11L12-038_A1.RNA_MOSCOT_paired_hvg.h5ad")
adata_rna

In [ ]:
adata_msi = sc.read_h5ad("/lustre/groups/ml01/workspace/eirini.giannakoulia/datasets/V11L12-038/V11L12-038_A1/V11L12-038_A1.MSI_MOSCOT_paired_hvg.h5ad")
adata_msi

In [ ]:
adata_rna_train = adata_rna[adata_rna.obs["split"] == "train"]
adata_rna_test = adata_rna[adata_rna.obs["split"] == "test"]
adata_msi_train = adata_msi[adata_msi.obs["split"] == "train"]
adata_msi_test = adata_msi[adata_msi.obs["split"] == "test"]

In [ ]:
coords_rna_train = adata_rna_train.obsm["spatial_warp"]
coords_rna_test = adata_rna_test.obsm["spatial_warp"]

In [ ]:
class MultiLayerGCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=3, dropout=0.5):
        super(MultiLayerGCN, self).__init__()
        
        # Define GCN layers
        self.layers = torch.nn.ModuleList()
        self.layers.append(GCNConv(input_dim, hidden_dim))  # First layer
        
        # Add hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(GCNConv(hidden_dim, hidden_dim))  # Middle layers
        
        self.output_layer = GCNConv(hidden_dim, output_dim)  # Output layer
        
        self.dropout = dropout

    def forward(self, x, edge_index):
        # Pass data through each GCN layer with ReLU activation
        for layer in self.layers:
            x = layer(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        
        # Final GCN layer (output layer)
        x = self.output_layer(x, edge_index)
        
        return x  